In [1]:
import itertools
import pathlib
import re

import numpy as np
import pandas as pd
import requests

In [2]:
def process_edges_to_full_network(edges_df, mapping, allow_loop=False, directed=False):
    '''
    Convert a DataFrame that contains only node pairs where an edge appears in one of 
    the three networks (train, test_recon, test_new) to a DataFrame of all node pairs 
    for nodes appearing in the test network.
    
    Parameters
    ----------
    edges_df : pandas.DataFrame
        Columns should be ['name_a', 'name_b', 'id_a', 'id_b', 'train', 'test_recon', 'test_new'].
        Contains only node pairs with at least one edge, ie. at minimum one of train, test_recon, 
        test_new is 1.
    mapping : dict
        Mapping from name to id
    allow_loop : bool
        Whether to include self-loops as potential edges.
    directed : bool
        Whether edge (a, b) is equivalent to (b, a). If directed, then only source-target node pairs
        will be in the returned DataFrame
        
    Returns
    -------
    pandas.DataFrame
    '''
    reversed_mapping = {v: k for k, v in mapping.items()}
    train_df = edges_df.query('train == 1')
    rel = '<=' if allow_loop else '<'
    
    if directed:
        source_nodes = sorted(set(train_df['id_a']))
        target_nodes = sorted(set(train_df['id_b']))
        id_a, id_b = zip(*itertools.product(source_nodes, target_nodes))
    else:
        nodes = sorted(set(train_df[['id_a', 'id_b']].values.flatten()))
        id_a, id_b = zip(*itertools.product(nodes, nodes))
    
    df = (
        pd.DataFrame()
        .assign(
            id_a=id_a,
            id_b=id_b,
        )
        .query(f'id_a {rel} id_b')
        .merge(edges_df, how='left', on=['id_a', 'id_b'])
        .assign(
            name_a=lambda df: df['id_a'].map(reversed_mapping),
            name_b=lambda df: df['id_b'].map(reversed_mapping),
        )
        .fillna(0)
        .assign(
            train=lambda df: df['train'].astype(int),
            test_recon=lambda df: df['test_recon'].astype(int),
            test_new=lambda df: df['test_new'].astype(int),
        )
        .filter(items=['name_a', 'name_b', 'id_a', 'id_b', 'train', 'test_recon', 'test_new'])
    )
    return df

# 0. Setup folders

In [3]:
data_path = pathlib.Path('../data/')
data_path.joinpath('1.raw/').mkdir(exist_ok=True, parents=True)
data_path.joinpath('2.edges/').mkdir(exist_ok=True, parents=True)
data_path.joinpath('3.all_nodes/').mkdir(exist_ok=True, parents=True)
data_path.joinpath('4.data/').mkdir(exist_ok=True, parents=True)

# 1. Download raw files

### Protein-protein interaction networks

* STRING https://string-db.org/
* High-throughput, systematic PPIs
    * We use two networks from the same group, both created through high-throughput screening. Data is available for download at http://interactome.baderlab.org/download.
    * Rual et al. (2005) *Nature* https://www.ncbi.nlm.nih.gov/pubmed/16189514
    * Rolland et al. (2014) *Cell* https://www.ncbi.nlm.nih.gov/pubmed/25416956

### BioRxiv collaboration network

DOI: 10.1101/515643

Rxivist full database (doi:10.5281/zenodo.2566421) at https://zenodo.org/record/2566421


I used the following query to export a copy of the Rxivist BioRxiv scrape.

```sqlite
SELECT 
    aa.article, 
    art.title,
    auth.id as author_id, 
    auth.name as author_name, 
    auth.institution, 
    art.doi, 
    art.collection, 
    art.posted
FROM prod.article_authors aa
JOIN prod.authors auth
	ON aa.author = auth.id
JOIN prod.articles art
	ON aa.article = art.id
```

### Transcription-factor target-gene relationships

https://www.ncbi.nlm.nih.gov/pubmed/29325558

In [4]:
file_to_url = {
    'ppi_string.txt.gz': ('https://stringdb-static.org/download/protein.links.v11.0/'
                          '9606.protein.links.v11.0.txt.gz'),
    
    'ppi_string_mapping.tsv.gz': ('https://string-db.org/mapping_files/uniprot/'
                                  'human.uniprot_2_string.2018.tsv.gz'),
    
    'ppi_ht_1.psi': 'http://interactome.baderlab.org/data/Raul-Vidal(Nature_2005).psi',
    'ppi_ht_2.psi': 'http://interactome.baderlab.org/data/Rolland-Vidal(Cell_2014).psi',
    
    'tftg.gmt': ('https://static-content.springer.com/esm/art%3A10.1186%2Fs12915-017-0469-0/'
                 'MediaObjects/12915_2017_469_MOESM5_ESM.gmt')
}

# for file, url in file_to_url.items():
#     with open(data_path.joinpath(f'1.raw/{file}'), 'wb') as f:
#         res = requests.get(url)
#         f.write(res.content)

# 2. Process files to edges

Processing is generally as follows: 

(Note this example is for an undirected network with self-loops)

1. Convert raw relationship data (in whatever form) to 

| source 	| target 	| network A 	| network B 	|
|--------	|--------	|-----------	|-----------	|
| A      	| B      	| 1         	| 0         	|
| A      	| C      	| 1         	| 1         	|
| B      	| C      	| 0         	| 1         	|

2. Assign 70% of Network1 edges to the training network. Map the nodes in the training network to the integers 0, ..., num(nodes)-1. If the network is undirected, ensure that `id_a` $\leq$ `id_b`. If the network is directed, index the source nodes first, (0, ..., num(source)-1), then target nodes (num(source),...). This mapping is done for the convenience of XSwap later. Results in `[network name]_edges_df`, which have the following schema:

| source 	| target 	| source_id 	| target_id 	| train 	| network A 	| network B 	|
|--------	|--------	|-----------	|-----------	|-------	|-----------	|-----------	|
| A      	| B      	| 0         	| 1         	| 0     	| 1         	| 0         	|
| A      	| C      	| 0         	| 2         	| 1     	| 1         	| 1         	|
| B      	| C      	| 1         	| 2         	| 0     	| 0         	| 1         	|

3. Take the subset of nodes that have an edge in the training network. The Cartesian product of these nodes will be the `[network_name]_df`, which have the following schema:

| source 	| target 	| source_id 	| target_id 	| train 	| network A 	| network B 	|
|--------	|--------	|-----------	|-----------	|-------	|-----------	|-----------	|
| A      	| A      	| 0         	| 0         	| 0     	| 0         	| 0         	|
| A      	| B      	| 0         	| 1         	| 0     	| 1         	| 0         	|
| A      	| C      	| 0         	| 2         	| 1     	| 1         	| 1         	|
| B      	| B      	| 1         	| 1         	| 0     	| 0         	| 0         	|
| B      	| C      	| 1         	| 2         	| 0     	| 0         	| 1         	|
| C      	| C      	| 2         	| 2         	| 0     	| 0         	| 0         	|


## 2.1 PPI

### 2.1.1 STRING

The two PPI networks use different mappings. We convert STRING to UniProt identifiers.

In [5]:
# Ensembl to UniProtKB identifier mappings
mapping_df = pd.read_table(data_path.joinpath('1.raw/ppi_string_mapping.tsv.gz'), 
                           compression='gzip', names=['species', 'uniprot_entry', 'string', 
                                                      'unknown_a', 'unknown_b'])

# Create dictionary with mappings
string_to_uniprot = (
    mapping_df
    .assign(uniprot=lambda df: df['uniprot_entry'].apply(lambda x: re.search('[A-Z0-9]+', x).group()))
    .set_index('string')
    .loc[:, 'uniprot']
    .to_dict()
)

# Load PPI network from STRING
string_edges = set(map(tuple, map(sorted,
    pd.read_table(data_path.joinpath('1.raw/ppi_string.txt.gz'), compression='gzip', 
                  sep=' ', dtype=str)
    .assign(
        uniprot_a=lambda df: df['protein1'].map(string_to_uniprot),
        uniprot_b=lambda df: df['protein2'].map(string_to_uniprot),
    )
    .dropna()
    .loc[:, ['uniprot_a', 'uniprot_b']]
    .values
)))

string_edges_df = (
    pd.DataFrame(sorted(string_edges), columns=['uniprot_a', 'uniprot_b'])
    .assign(
        test_recon=1,
    )
)

string_edges_df.head(2)

,uniprot_a,uniprot_b,test_recon
0,A0A024R161,A0A075B734,1
1,A0A024R161,A2A3L6,1


### 2.1.2 High-throughput PPI network

In [6]:
# Combine the two networks
ht_df = pd.concat([
    pd.read_csv(data_path.joinpath('1.raw/ppi_ht_1.psi'), sep='\t'), 
    pd.read_csv(data_path.joinpath('1.raw/ppi_ht_2.psi'), sep='\t')
], ignore_index=True)

ht_edges = set(map(tuple, map(sorted, 
    ht_df
    .rename(columns={
        'Unique identifier for interactor A': 'ida', 
        'Unique identifier for interactor B': 'idb'})
    .filter(items=['ida', 'idb',])
    .query('ida != "-" and idb != "-"')
    .assign(
        uniprot_a=lambda df: df['ida'].apply(lambda x: re.search('(?<=uniprotkb:)[0-9A-Z]+', x).group()),
        uniprot_b=lambda df: df['idb'].apply(lambda x: re.search('(?<=uniprotkb:)[0-9A-Z]+', x).group()),
    )
    .loc[:, ['uniprot_a', 'uniprot_b']]
    .values
)))

ht_edges_df = (
    pd.DataFrame(sorted(ht_edges), columns=['uniprot_a', 'uniprot_b'])
    .assign(
        test_new=1,
    )
)

ht_edges_df.head(2)

,uniprot_a,uniprot_b,test_new
0,A0A024R0Y4,A0A0R4J2E4,1
1,A0A024R0Y4,O14964,1


### 2.1.3 Combined PPI network

Now, having two PPI networks both mapped to UniProt identifiers, we subset to the intersection of the two sets of nodes, using only nodes that are present in both networks. Then we map the shared nodes to IDs, unique integers from 0 to the number of shared nodes. This is done for efficiency in XSwap later on. Finally, as the edges are undirected, they are sorted so that the first ID is always <= the second ID. This ensures that we don't accidentally miss duplicates, etc.

In [7]:
# Only use nodes that are present in both networks
string_nodes = set(string_edges_df.loc[:, 'uniprot_a':'uniprot_b'].values.flatten())
ht_nodes = set(ht_edges_df.loc[:, 'uniprot_a':'uniprot_b'].values.flatten())
shared_nodes = set(string_nodes.intersection(ht_nodes))

print(f'STRING: {len(string_nodes)} nodes\nHT: {len(ht_nodes)} nodes\n'
      f'SHARED: {len(shared_nodes)} nodes')

# Join DataFrames and subset to node pairs consisting only of nodes shared between both networks
np.random.seed(0)
ppi_edges_df = (
    string_edges_df  # ERROR COULD BE IN OUTER JOIN
    .merge(ht_edges_df, how='outer', on=['uniprot_a', 'uniprot_b'])
    .loc[lambda df: (df['uniprot_a'].apply(lambda x: x in shared_nodes) & 
                     df['uniprot_b'].apply(lambda x: x in shared_nodes))]
    .fillna(0)
    .assign(
        train=lambda df: df['test_recon'].apply(lambda x: x == 1 and np.random.rand() < 0.7).astype(int),
        test_recon=lambda df: df['test_recon'].astype(int),
        test_new=lambda df: df['test_new'].astype(int),
    )
)

# Map nodes onto unique integers (for XSwap)
ppi_nodes = sorted(set(
    ppi_edges_df
    .query('train == 1')
    .loc[:, 'uniprot_a':'uniprot_b']
    .values.flatten()
))
ppi_mapping = {name: i for name, i in zip(ppi_nodes, range(len(ppi_nodes)))}
ppi_reversed_mapping = {v: k for k, v in ppi_mapping.items()}

# Create a DF of all edges whose nodes have an edge in at least one of the networks
ppi_edges_df = (
    ppi_edges_df
    .assign(
        mapped_a=lambda df: df['uniprot_a'].map(ppi_mapping),
        mapped_b=lambda df: df['uniprot_b'].map(ppi_mapping),
    )
    # Drop node pairs with nodes not in the train network
    .dropna()
    .assign(
        # Edges are bi-directional, so make id_a <= id_b
        id_a=lambda df: df.apply(lambda row: min(row['mapped_a'], row['mapped_b']), axis=1).astype(int),
        id_b=lambda df: df.apply(lambda row: max(row['mapped_a'], row['mapped_b']), axis=1).astype(int),
        
        # Re-ordering means that UniProt IDs may now be reversed. 
        # Apply reverse mapping to ensure correctness.
        name_a=lambda df: df['id_a'].map(ppi_reversed_mapping),
        name_b=lambda df: df['id_b'].map(ppi_reversed_mapping),
    )
    .filter(items=['name_a', 'name_b', 'id_a', 'id_b', 'train', 'test_recon', 'test_new'])
    .reset_index(drop=True)
)

ppi_edges_df.to_csv(data_path.joinpath('2.edges/ppi.tsv.xz'), compression='xz', index=False, sep='\t')

ppi_edges_df.head(2)

STRING: 19080 nodes
HT: 4517 nodes
SHARED: 4083 nodes


,name_a,name_b,id_a,id_b,train,test_recon,test_new
0,A0A087WT00,O00154,0,48,1,1,0
1,A0A087WT00,O43736,0,237,0,1,0


In [8]:
ppi_df = process_edges_to_full_network(ppi_edges_df, ppi_mapping, allow_loop=True, directed=False)

ppi_df.to_csv(data_path.joinpath('3.all_nodes/ppi.tsv.xz'), compression='xz', index=False, sep='\t')

ppi_df.head(2)

,name_a,name_b,id_a,id_b,train,test_recon,test_new
0,A0A087WT00,A0A087WT00,0,0,0,0,0
1,A0A087WT00,A0A0B4J1W7,0,1,0,0,0


## 2.2 BioRxiv collaboration network

In [10]:
rxivist_df = (
    pd.read_csv('../data/1.raw/citations.csv.xz', compression='xz')
    .dropna()
    .assign(
        year=lambda df: df['posted'].apply(lambda x: int(x[:4])),
    )
    .query('collection == "bioinformatics"')
    .drop(columns=['title', 'author_id', 'institution', 'doi', 'posted', 'collection'])
)

rxivist_df.head(2)

,article,author_name,year
14114,2146,Juri Rappsilber,2018
14115,2146,Swantje Lenz,2018


In [11]:
train_cutoff = 2017

# Self-join to create author-author relationships from author-article relationships
biorxiv_edges_df = (
    rxivist_df
    .merge(rxivist_df, on=['article', 'year'], )
    .rename(columns={'author_name_x': 'name_a', 'author_name_y': 'name_b'})
    # Remove duplicate and self-edges
    .query('name_a < name_b')
    .assign(
        test_recon=lambda df: df['year'].apply(lambda x: x <= train_cutoff).astype(int),
        test_new=1,
    )
)

# Select articles for the training network
articles = sorted(set(biorxiv_edges_df.query('test_recon == 1')['article']))
train_articles = set(np.random.choice(articles, size=int(0.7*len(articles)), replace=False))
biorxiv_edges_df = (
    biorxiv_edges_df
    .assign(train=lambda df: df['article'].apply(lambda x: x in train_articles).astype(int))
)

# Create a node mapping
authors = sorted(set(biorxiv_edges_df.query('train == 1').loc[:, ['name_a', 'name_b']].values.flatten()))
biorxiv_mapping = {name: i for name, i in zip(authors, range(len(authors)))}
biorxiv_reversed = {v: k for k, v in biorxiv_mapping.items()}

# Apply node mapping
biorxiv_edges_df = (
    biorxiv_edges_df
    .assign(
        mapped_a=lambda df: df['name_a'].map(biorxiv_mapping),
        mapped_b=lambda df: df['name_b'].map(biorxiv_mapping),
    )
    .dropna()
    .assign(
        id_a=lambda df: df.apply(lambda row: min(row['mapped_a'], row['mapped_b']), axis=1).astype(int),
        id_b=lambda df: df.apply(lambda row: max(row['mapped_a'], row['mapped_b']), axis=1).astype(int),
        name_a=lambda df: df['id_a'].map(biorxiv_reversed),
        name_b=lambda df: df['id_b'].map(biorxiv_reversed),
    )
    .reset_index(drop=True)
    .filter(items=['name_a', 'name_b', 'id_a', 'id_b', 'train', 'test_recon', 'test_new'])
)

biorxiv_edges_df.to_csv(data_path.joinpath('2.edges/biorxiv.tsv.xz'), compression='xz', index=False, sep='\t')

biorxiv_edges_df.head(2)

,name_a,name_b,id_a,id_b,train,test_recon,test_new
0,Juri Rappsilber,Lutz Fischer,3607,4175,0,0,1
1,Kemal Eren,Venkatesh Kumar,3754,6944,0,0,1


In [12]:
biorxiv_df = process_edges_to_full_network(biorxiv_edges_df, biorxiv_mapping, allow_loop=False, directed=False)

biorxiv_df.to_csv(data_path.joinpath('3.all_nodes/biorxiv.tsv.xz'), compression='xz', index=False, sep='\t')

biorxiv_df.head(2)

,name_a,name_b,id_a,id_b,train,test_recon,test_new
0,- The US-Venezuela Collaborative Research Group,A. Ercument Cicek,0,1,0,0,0
1,- The US-Venezuela Collaborative Research Group,A. S. M. Ashique Mahmood,0,2,0,0,0


## 2.3 Transcription factor - target gene (TFTG) network

Data is originally in the form:

Source: [target1, target2, ...]

and needs to first be reformatted as an edge list.

In [14]:
# Format data to edges
tftg_records = list()
with open(data_path.joinpath('1.raw/tftg.gmt'), 'r') as f:
    for line in f.readlines():
        groups = line.strip().split('\t')
        tf_name, tf_entrez = groups[0].split('_')[-2:]
        method = re.match('\A([A-Za-z\-\ ]+)(?=(\ Transcriptional|\ TFTG))', groups[1]).group().lower()
        for gene in groups[2:]:
            tftg_records.append(
                (tf_name, gene, method)
            )

# Format edges to a DataFrame. Randomly assign 70% of low-throughput edges to train, with
# the withheld 30% being used for predicting reconstruction.
np.random.seed(0)
tftg_edges_df = (
    pd.DataFrame
    .from_records(tftg_records, columns=['name_a', 'name_b', 'method'])
    .assign(
        test_recon=lambda df: (df['method'] == 'low throughtput').astype(int),
        test_new=lambda df: (df['method'] == 'chip-seq').astype(int),
    )
    .groupby(['name_a', 'name_b'])[['test_recon', 'test_new']].sum()
    .reset_index()
    .assign(
        train=lambda df: df['test_recon'].apply(lambda x: x == 1 and np.random.rand() < 0.7).astype(int),
    )
)

# Create a mapping between nodes and integers. Make TFs the lowest values, and non-TF genes later
tfs = set(tftg_edges_df.query('train == 1').loc[:, 'name_a'].values)
genes = set(tftg_edges_df.query('train == 1').loc[:, 'name_b'].values)
genes_only = sorted(genes.difference(tfs))

tfs = sorted(tfs)
genes = sorted(genes)

tf_mapping = {tf: i for i, tf in enumerate(tfs)}
gene_mapping = {gene: len(tfs) + i for i, gene in enumerate(genes_only)}
tftg_mapping = {**tf_mapping, **gene_mapping}

print("{} unique transcription factors\n{} unique genes".format(len(tfs), len(genes)))

tftg_edges_df = (
    tftg_edges_df
    .assign(
        id_a=lambda df: df['name_a'].map(tftg_mapping),
        id_b=lambda df: df['name_b'].map(tftg_mapping),
    )
    .dropna()
    .assign(
        id_a=lambda df: df['id_a'].astype(int),
        id_b=lambda df: df['id_b'].astype(int),
    )
    .filter(items=['name_a', 'name_b', 'id_a', 'id_b', 'train', 'test_recon', 'test_new'])
)

tftg_edges_df.to_csv(data_path.joinpath('2.edges/tftg.tsv.xz'), compression='xz', index=False, sep='\t')

tftg_edges_df.head(2)

232 unique transcription factors
14913 unique genes


,name_a,name_b,id_a,id_b,train,test_recon,test_new
0,AEBP2,AAGAB,498,239,0,0,1
1,AEBP2,ALDH4A1,498,613,0,0,1


In [15]:
tftg_df = process_edges_to_full_network(tftg_edges_df, tftg_mapping, allow_loop=True, directed=True)

tftg_df.to_csv(data_path.joinpath('3.all_nodes/tftg.tsv.xz'), compression='xz', index=False, sep='\t')

tftg_df.head(2)

,name_a,name_b,id_a,id_b,train,test_recon,test_new
0,AHR,AHR,0,0,0,0,0
1,AHR,AR,0,1,0,0,0
